In [ ]:
# 匯入「keras_lite_convertor.py」到 Colab

from google.colab import files 
uploaded = files.upload()       # 匯入檔案

Saving keras_lite_convertor.py to keras_lite_convertor.py


In [ ]:
# 讀取 gesture_data 資料夾內的檔案

import keras_lite_convertor as kc

path_name = 'gesture_data'    
Data_reader = kc.Data_reader(path_name,
                             mode='categorical',    # categorical 適用於多元分類
                             label_name = ['right','down','stop',   # 標籤名稱
                                           'left','up'])   
data, label = Data_reader.read()  

In [ ]:
# 標籤   [1 0 0 0 0]代表 right、[0 1 0 0 0]代表 down...用上面的標籤名稱以此類推

label[:3]

[[0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]


In [ ]:
# 資料預處理

# 取資料中的 90% 當作訓練集
split_num = int(len(data)*0.9) 
train_data=data[:split_num]
train_label=label[:split_num]    

# 正規化
mean = train_data.mean() # 平均數
data -= mean
std = train_data.std()   # 標準差
data /= std

In [ ]:
# 訓練集、驗證集、測試集的資料形狀

# 訓練集
print(train_data.shape)    

# 驗證集
validation_data=data[split_num:-5]  
print(validation_data.shape)
validation_label=label[split_num:-5]   

# 測試集
test_data=data[-5:]     
print(test_data.shape)
test_label=label[-5:]           

(450, 60)
(45, 60)
(5, 60)


In [ ]:
# 建立神經網路架構

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

# 建立神經網路物件
model = Sequential()  

# 輸出層的啟動函數為 softmax(輸出值介於 0~1 之間, 總和為 1, 適合用於分類的神經網路)
model.add(layers.Dense(label.shape[1],activation='softmax',input_shape=(60,))) 

# 顯示模型資訊
model.summary()  

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 5)                 305       
Total params: 305
Trainable params: 305
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 編譯及訓練模型

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
train_history = model.fit(train_data,train_label,                              # 訓練集
                          validation_data=(validation_data,validation_label),  # 驗證集
                          epochs=300)   # 訓練週期

Epoch 1/300
15/15 [==============================] - 0s 15ms/step - loss: 2.9785 - acc: 0.0333 - val_loss: 2.6887 - val_acc: 0.0444
Epoch 2/300
15/15 [==============================] - 0s 4ms/step - loss: 2.4902 - acc: 0.0867 - val_loss: 2.2243 - val_acc: 0.1556
Epoch 3/300
15/15 [==============================] - 0s 4ms/step - loss: 2.0955 - acc: 0.1689 - val_loss: 1.8494 - val_acc: 0.2667
Epoch 4/300
15/15 [==============================] - 0s 4ms/step - loss: 1.7645 - acc: 0.2889 - val_loss: 1.5453 - val_acc: 0.4000
Epoch 5/300
15/15 [==============================] - 0s 3ms/step - loss: 1.5104 - acc: 0.3822 - val_loss: 1.3132 - val_acc: 0.4889
Epoch 6/300
15/15 [==============================] - 0s 4ms/step - loss: 1.3088 - acc: 0.4867 - val_loss: 1.1423 - val_acc: 0.6222
Epoch 7/300
15/15 [==============================] - 0s 4ms/step - loss: 1.1534 - acc: 0.6067 - val_loss: 1.0111 - val_acc: 0.7333
Epoch 8/300
15/15 [==============================] - 0s 4ms/step - loss: 1.0310 - 

In [ ]:
# 測試模型

# 預測值
print('predict:')
print(model.predict(test_data))
print()
# 實際值
print('real:')
print(test_label)

predict:
[[6.0836935e-14 1.6169720e-06 4.1332806e-04 9.9956793e-01 1.7209695e-05]
 [2.4066780e-05 1.6504347e-03 3.7742618e-02 9.5945334e-01 1.1296043e-03]
 [1.0026272e-05 1.2925485e-09 2.1276509e-03 2.1029995e-05 9.9784124e-01]
 [9.9996412e-01 5.3112194e-07 2.9978386e-05 2.5448899e-09 5.4535585e-06]
 [5.8928271e-07 9.9994326e-01 5.6050769e-05 7.8687627e-08 1.0252019e-11]]

real:
[[0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]]


In [ ]:
# 儲存模型

kc.save(model,'gesture_model.json')

In [ ]:
# 顯示資訊

print('mean=', mean)  # 平均值
print('std=', std)    # 標準差

mean= 2642.4535925925925
std= 10607.848009804136
